In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import os

In [3]:
import os
import pandas as pd

def replace_emotion_pos(folder_path):
    # List all CSV files in the specified folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Initialize an empty DataFrame to store the merged data
    merged_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        # Read the CSV file into a DataFrame
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        # Replace 'emotion' with 'positive' in the 'state' column
        df.loc[df['state'] == 'emotion', 'state'] = 'positive'

        # Concatenate the current DataFrame with the merged DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv('merged_positive_data.csv', index=False)


def replace_emotion_neg(folder_path):
    # List all CSV files in the specified folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Initialize an empty DataFrame to store the merged data
    merged_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        # Read the CSV file into a DataFrame
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        # Replace 'emotion' with 'negative' in the 'state' column
        df.loc[df['state'] == 'emotion', 'state'] = 'negative'

        # Concatenate the current DataFrame with the merged DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv('merged_negative_data.csv', index=False)

if __name__ == "__main__":
    folder_path_pos = '/Users/alexisbader/Desktop/446 Data ALL DATA/Positive'  # Specify the folder containing the CSV files
    folder_path_neg = '/Users/alexisbader/Desktop/446 Data ALL DATA/Positive'  # Specify the folder containing the CSV files
    replace_emotion_pos(folder_path_pos)
    replace_emotion_neg(folder_path_neg)


In [4]:
import pandas as pd

def process_csv(positive_file, negative_file):
    # Read the positive and negative CSV files into DataFrames
    df_positive = pd.read_csv(positive_file)
    df_negative = pd.read_csv(negative_file)

    # Create new columns for positive, negative, and neutral in the positive DataFrame
    df_positive['positive'] = df_positive['state'].apply(lambda x: 1 if x == 'emotion' else 0)
    df_positive['negative'] = df_positive['state'].apply(lambda x: 1 if x == 'negative' else 0)
    df_positive['neutral'] = df_positive['state'].apply(lambda x: 1 if x == 'neutral' else 0)

    # Create new columns for positive, negative, and neutral in the negative DataFrame
    df_negative['positive'] = df_negative['state'].apply(lambda x: 1 if x == 'positive' else 0)
    df_negative['negative'] = df_negative['state'].apply(lambda x: 1 if x == 'emotion' else 0)
    df_negative['neutral'] = df_negative['state'].apply(lambda x: 1 if x == 'neutral' else 0)

    # Concatenate positive and negative DataFrames into a single DataFrame
    merged_df = pd.concat([df_positive, df_negative], ignore_index=True)

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv('merged_data.csv', index=False)

if __name__ == "__main__":
    positive_file_path = 'merged_positive_data.csv'  # Specify the path to the positive CSV file
    negative_file_path = 'merged_negative_data.csv'  # Specify the path to the negative CSV file

    # Process the positive and negative CSV files and merge the results
    process_csv(positive_file_path, negative_file_path)


In [5]:
# Preprocessing
df = pd.read_csv('merged_data.csv') # Converting to dataframe

# Dropping neutral data
mask = df['state'] != 'neutral'
df = df.drop(df.index[~mask])

# Encoding nominal data, dropping outdated column names
state_mapping = {'negative': 0, 'positive': 1}
df['encoded_state'] = df['state'].map(state_mapping)

df = df.drop(['positive', 'negative', 'neutral'], axis=1)

# print data(feature)shape
data = df.drop(['state', 'label', 'encoded_state'], axis=1)
target = df['encoded_state']


In [7]:
# Split dataset into training set and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,random_state=109) # 70% training and 30% test

In [14]:
# Testing SVM performance
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
svm_pred = clf.predict(X_test)
print("SVM Accuracy:",metrics.accuracy_score(y_test, svm_pred))


In [17]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model = LogisticRegression(solver='liblinear', penalty='l1')
model.fit(X_train, y_train)

lg_pred = model.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, lg_pred))

Logistic Regression Accuracy: 0.8192090395480226


In [18]:
# NB Classification
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

nb_pred = model_nb.predict(X_test)
print("NB Accuracy:",metrics.accuracy_score(y_test, nb_pred))

NB Accuracy: 0.6022598870056497


In [19]:
# KNN Classification
from sklearn.neighbors import KNeighborsClassifier

knn_accuracies = {'k': [], 'accuracy': []}
for i in range(1, 20):
    model_knn = KNeighborsClassifier(n_neighbors=i)
    model_knn.fit(X_train, y_train)
    knn_accuracies['k'].append(i)
    y_pred4 = model_knn.predict(X_test)
    knn_accuracies['accuracy'].append(metrics.accuracy_score(y_test, y_pred4))

# Calculate the accuracy score
print(knn_accuracies) # k=5 is best

/Users/navalan/Documents/GitHub/BCIEmotion/venv/lib/python3.8/site-packages/threadpoolctl.py:1195: RuntimeWarning: libc not found. The ctypes module in Python 3.8 is maybe too old for this OS.
  warnings.warn(


{'k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'accuracy': [0.7480225988700565, 0.7163841807909604, 0.7672316384180791, 0.7344632768361582, 0.7717514124293785, 0.7638418079096045, 0.7694915254237288, 0.7559322033898305, 0.7536723163841808, 0.7559322033898305, 0.7502824858757062, 0.7536723163841808, 0.7638418079096045, 0.7446327683615819, 0.7593220338983051, 0.7446327683615819, 0.7536723163841808, 0.7401129943502824, 0.7548022598870057]}


In [10]:
## ANN Classifaction
import numpy as np
from tensorflow import keras

ann = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(40,)), # Input layer has 40 features
    keras.layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 neuron (binary classification)
])

ann.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

test_loss, test_accuracy = ann.evaluate(X_test, y_test)
ann.summary()
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/30
178/178 [==============================] - 1s 2ms/step - loss: 1.3091 - accuracy: 0.6868 - val_loss: 0.4661 - val_accuracy: 0.7968
Epoch 2/30
178/178 [==============================] - 0s 1ms/step - loss: 0.5797 - accuracy: 0.7837 - val_loss: 0.4001 - val_accuracy: 0.8250
Epoch 3/30
178/178 [==============================] - 0s 1ms/step - loss: 0.5309 - accuracy: 0.8103 - val_loss: 0.3909 - val_accuracy: 0.8299
Epoch 4/30
178/178 [==============================] - 0s 1ms/step - loss: 0.3972 - accuracy: 0.8407 - val_loss: 0.3300 - val_accuracy: 0.8462
Epoch 5/30
178/178 [==============================] - 0s 1ms/step - loss: 0.3686 - accuracy: 0.8474 - val_loss: 0.3192 - val_accuracy: 0.8666
Epoch 6/30
178/178 [==============================] - 0s 1ms/step - loss: 0.3830 - accuracy: 0.8474 - val_loss: 0.3669 - val_accuracy: 0.8560
Epoch 7/30
178/178 [==============================] - 0s 1ms/step - loss: 0.3396 - accuracy: 0.8594 - val_loss: 0.2791 - val_accuracy: 0.8730
Epoch 

In [29]:
import numpy as np
y_pred = svm_pred

np.unique(y_pred)

xx = np.where(y_pred == y_test)[0]
corr = y_test.iloc[xx]
wrong = y_test.iloc[~xx]
vals_right, counts_right = np.unique(corr, return_counts=True)
vals_wrong, counts_wrong = np.unique(wrong, return_counts=True)
vals_real, counts_real = np.unique(y_test, return_counts=True)

for i, val in enumerate(vals_right):
    print(f"accuracy val: {val} count: {counts_right[i]}/{counts_real[i]} = {counts_right[i]/counts_real[i] * 100}%")
for i, val in enumerate(vals_wrong):
    print(f"val wrong: {val} count: {counts_wrong[i]}")



accuracy val: 0 count: 372/426 = 87.32394366197182%
accuracy val: 1 count: 352/459 = 76.68845315904139%
val wrong: 0 count: 340
val wrong: 1 count: 384


In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming you have y_pred and y_test
# y_pred: Predicted labels
# y_test: True labels

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

class_labels = sorted(set(y_test))
print("Confusion Matrix:")
print(pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extract true positives, false positives, true negatives, and false negatives
true_positives = np.diag(conf_matrix)
false_positives = np.sum(conf_matrix, axis=0) - true_positives
false_negatives = np.sum(conf_matrix, axis=1) - true_positives
true_negatives = np.sum(conf_matrix) - (true_positives + false_positives + false_negatives)

# Display additional metrics
print("\nTrue Positives:")
print(true_positives)

print("\nFalse Positives:")
print(false_positives)

print("\nFalse Negatives:")
print(false_negatives)

print("\nTrue Negatives:")
print(true_negatives)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

Confusion Matrix:
     0    1
0  372   54
1  107  352
Confusion Matrix:
[[372  54]
 [107 352]]

True Positives:
[372 352]

False Positives:
[107  54]

False Negatives:
[ 54 107]

True Negatives:
[352 372]

Accuracy: 81.81%
